In [ ]:
from IPython.display import Image

import os
import numpy as np
import matplotlib.pyplot as plt

import flopy

In [ ]:
Image("./mf6ps1_domain.png")

### Create simulation and associated packages

1. Simulation
2. Temporal discretization package (TDIS)
3. Interative model solution package (IMS)

Set the model workspace for the model to `'flopy'` and use the name `'mf6ps1'` for the simulation and groundwater flow model.

In [ ]:
ws = "temp/simple"
name = "mf6ps1"
sim = flopy.mf6.MFSimulation(sim_name=name, exe_name='mf6', sim_ws=ws)

Add the TDIS package to the simulation. This will be a steady-state model with a single stress period so default values can be used.

In [ ]:
tdis = flopy.mf6.ModflowTdis(sim)

Add the IMS package to the simulation. Since this is a simple model the default IMS package values can be used.

In [ ]:
ims = flopy.mf6.ModflowIms(sim)

In [ ]:
Image("./mf6ps1_grid.png")

### Define the model properties

In [ ]:
nlay, nrow, ncol = 3, 21, 20
delr = delc = 500.
top = 400.
botm = [220., 200., 0.]
kh = [50., 0.01, 200.]
kv = [10., 0.01, 20.]

### Create the groundwater flow model and associated packages

1. Groundwater flow model (GWF)
2. Discretization package (DIS)
3. Node property flow package (NPF)
4. Initial conditions package (IC)
5. Constant head package (CHD)
6. Output control package (OC)

#### Create the groundwater flow model.

In [ ]:
gwf = flopy.mf6.ModflowGwf(sim, modelname=name)

#### Create the discretization package.

In [ ]:
dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc, top=top, botm=botm)

#### Create the npf file.

In [ ]:
npf = flopy.mf6.ModflowGwfnpf(gwf, k=kh, k33=kv)

#### Create the initial conditions package with an initial head of 400.

In [ ]:
ic = flopy.mf6.ModflowGwfic(gwf, strt=400)

#### Create a chd for the canal (left) boundary

In [ ]:
canal = [[0, i, 0, 330] for i in range(nrow)]
canal

In [ ]:
chdc = flopy.mf6.ModflowGwfchd(gwf, pname='canal', filename='canal.chd', stress_period_data={0: canal})

#### Create a chd for the river (right) boundary.

In [ ]:
river = [[0, i, ncol-1, 320] for i in range(nrow)]
river

In [ ]:
chdr = flopy.mf6.ModflowGwfchd(gwf, pname='river', filename='river.chd', stress_period_data={0: river})

#### Create an output control file and save the heads

In [ ]:
oc = flopy.mf6.ModflowGwfoc(gwf, head_filerecord=name+'.hds', saverecord=[['head', 'all']])

#### Write the model files

In [ ]:
sim.write_simulation()

#### Run the model

In [ ]:
sim.run_simulation()

#### Post process the results

In [ ]:
hds = gwf.output.head().get_data()

#### Plot the results

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf)
mm.plot_array(hds)
mm.plot_grid(lw=0.5, color='black')
mm.plot_bc(package=chdc, color='green')
mm.plot_bc(package=chdr, color='cyan')

#### Load the list file

In [ ]:
fpth = os.path.join(ws, name + ".lst")

In [ ]:
mflst = flopy.utils.Mf6ListBudget(fpth)

In [ ]:
mflst.get_dataframes()[0]

In [ ]:
mflst.get_dataframes()[1]